In [ ]:
amazon_PCA = loadtxt('/kaggle/input/amazon-kaggle-with-pca/Amazon_Kaggle_with_PCA.csv', delimiter=',', dtype=np.float64, skiprows=1)
meta3 = amazon_PCA[:,1:]
# Split data into training and testing sets
x_train3, x_test3, y_train3, y_test3 = train_test_split(amazon_PCA, output1, random_state=1)
y_train3 = np.ravel(y_train3)

max_score_PCA = 0
best_params_PCA = {}

# Perform grid search for best parameters for the PCA dataset
for n_estimators in param_grid['n_estimators']:
    for max_depth in param_grid['max_depth']:
        for min_samples_split in param_grid['min_samples_split']:
            for min_samples_leaf in param_grid['min_samples_leaf']:
                for max_features in param_grid['max_features']:
                    params = {
                        'n_estimators': n_estimators,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'max_features': max_features
                    }
                    forest_PCA = RandomForestClassifier(**params)
                    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
                    scores3 = cross_val_score(forest_PCA, x_train3, y_train3, cv=skf)
                    score_mean3 = scores3.mean()
                    if score_mean3 > max_score_PCA:
                        max_score_PCA = score_mean3
                        best_params_PCA = params

print("Best Parameters for PCA dataset:")
print(best_params_PCA)
print("Average Accuracy:", max_score_PCA)
f1_score_final3 = 0.0
prc_auc_final3 = 0.0
roc_auc_final3 = 0.0
y_test_all3 = []
y_pred_proba_all3 = []

for train_index, test_index in skf.split(x_train3, y_train3):
    x_train_fold3, x_val_fold3 = x_train3[train_index], x_train3[test_index]
    y_train_fold3, y_val_fold3 = y_train3[train_index], y_train3[test_index]
    
    y_train_fold3 = np.ravel(y_train_fold3)
    
    final_forest_PCA = RandomForestClassifier(**best_params_PCA)
    final_forest_PCA.fit(x_train_fold3, y_train_fold3)
    y_pred_final_PCA = final_forest_PCA.predict_proba(x_val_fold3)[:, 1]  # Predict probabilities
    
    y_test_all3.extend(y_val_fold3)
    y_pred_proba_all3.extend(y_pred_final_PCA)
    
    f1_final_PCA = f1_score(y_val_fold3, (y_pred_final_PCA > 0.5).astype(int))
    f1_score_final3 += f1_final_PCA
    
    precision3, recall3, _ = precision_recall_curve(y_val_fold3.ravel(), y_pred_final_PCA.ravel())
    prc_auc3 = average_precision_score(y_val_fold3.ravel(), y_pred_final_PCA.ravel())
    prc_auc_final3 += prc_auc3
    
fpr_final3, tpr_final3, _ = roc_curve(y_test_all3, y_pred_proba_all3)
roc_auc_final3 = roc_auc_score(y_test_all3, y_pred_proba_all3)

f1_score_final3 /= skf.n_splits
prc_auc_final3 /= skf.n_splits

conf_matrix_final3 = confusion_matrix(y_test_all3, (np.array(y_pred_proba_all3) > 0.5).astype(int))

tpr_final_value3 = conf_matrix_final3[1, 1] / (conf_matrix_final3[1, 1] + conf_matrix_final3[1, 0])
fpr_final_value3 = conf_matrix_final3[0, 1] / (conf_matrix_final3[0, 1] + conf_matrix_final3[0, 0])

print("F1 Score for PCA dataset:", f1_score_final3)
print("Precision for PCA dataset: ", precision3)
print("Recall for PCA dataset:", recall3)
print("Precision-Recall AUC for PCA dataset:", prc_auc_final3)
print("ROC AUC for PCA dataset:", roc_auc_final3)
print("TPR for PCA roc:", tpr_final3)
print("FPR for PCA roc:", fpr_final3)
print("TPR for PCA:", tpr_final_value3)
print("FPR for PCA:", fpr_final_value3)

plt.figure()
plt.plot(fpr_final3, tpr_final3, color='darkorange', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc_final3)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve for PCA')
plt.legend(loc="lower right")
plt.savefig('/kaggle/working/roc_curve_pca.png')
plt.show()

plt.figure()
plt.plot(recall3, precision3, color='blue', lw=2, label='Precision-Recall curve (AUC = %0.2f)' % prc_auc_final3)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for PCA')
plt.legend(loc="lower left")
plt.savefig('/kaggle/working/precision_recall_curve_pca.png')
plt.show()